## Install Requirements

In [1]:
# pip install python-dotenv

## Import Modules

In [2]:
from dotenv import load_dotenv
load_dotenv()
import requests
import json
import os
from datetime import datetime
import pandas as pd

## Define Constants

In [3]:
## Constants
POSITION_MAP = {
    0: 'UNKNOWN',
    1: 'QB',
    2: 'RB',
    3: 'WR',
    4: 'TE',
    5: 'K',
    6: 'UNKNOWN',
    7: 'OP',
    8: 'DT',
    9: 'DE',
    10: 'LB',
    11: 'DL',
    12: 'CB',
    13: 'S',
    14: 'DB',
    15: 'DP',
    16: 'D/ST',
    17: 'UNKNOWN',
    18: 'P',
    19: 'HC',
    20: 'BE',
    21: 'IR',
    22: '',
    23: 'RB/WR/TE',
    24: 'ER',
    25: 'Rookie',
    'QB': 0,
    'RB': 2,
    'WR': 4,
    'TE': 6,
    'D/ST': 16,
    'K': 17,
    'FLEX': 23,
    'DT': 8,
    'DE': 9,
    'LB': 10,
    'DL': 11,
    'CB': 12,
    'S': 13,
    'DB': 14,
    'DP': 15,
    'HC': 19
}

## Define League Info Variables

In [4]:
league_id = os.environ.get('LEAGUE_ID')
cookie_swid = os.environ.get('SWID')
cookie_espn_s2 = os.environ.get('ESPN_S2')

## Get Player Base

In [5]:
def get_player_base(year):
    player_base = {}

    endpoint = f'https://fantasy.espn.com/apis/v3/games/ffl/seasons/{year}/players'
    params = {'view': 'players_wl'}
    filters = {'filterActive': {'value': True}}
    headers = {'x-fantasy-filter': json.dumps(filters)}
    cookies = {"swid":      cookie_swid,
               "espn_s2":   cookie_espn_s2}

    response = requests.get(endpoint, params=params, headers=headers, cookies=cookies)
    print(response.url)

    for player in response.json():
        player_id = player['id']
        player_full_name = player['fullName']
        player_position = POSITION_MAP[player['defaultPositionId']]

        player_base[player_id] = (player_full_name, player_position)

        if player_full_name not in player_base:
            player_base[player_full_name] = (player_id, player_position)
    
    return player_base

In [6]:
## Test Cases for Positional Data
for year in range(2018, 2023):
    player_base = get_player_base(year)
    assert player_base['DeAndre Hopkins'][1] == 'WR'
    assert player_base['Tom Brady'][1] == 'QB'
    assert player_base['Sony Michel'][1] == 'RB'
    assert player_base['Zach Ertz'][1] == 'TE'
    assert player_base['Robbie Gould'][1] == 'K'

    for player in player_base:
        if player_base[player][1] == 'UNKNOWN':
            print(player_base[player])
        # print(player_base[player])

https://fantasy.espn.com/apis/v3/games/ffl/seasons/2018/players?view=players_wl
https://fantasy.espn.com/apis/v3/games/ffl/seasons/2019/players?view=players_wl
https://fantasy.espn.com/apis/v3/games/ffl/seasons/2020/players?view=players_wl
https://fantasy.espn.com/apis/v3/games/ffl/seasons/2021/players?view=players_wl
https://fantasy.espn.com/apis/v3/games/ffl/seasons/2022/players?view=players_wl


## Build Offers Dataframe

In [7]:
# transaction_id          : transaction['id']
# year                    : year
# scoring_period          : scoring_period
# proposed_date           : transaction['proposedDate']
# processed_date          : transaction['processedDate']
# execution type          : transaction['executionType']
# transaction_type        : transaction['type']
# Team Owner              : teams[transaction['memberId']]
# status                  : transaction['status']
# bid amount              : transaction['bidAmount']
# item_type               : transaction['items'][x]['type']
# fromTeamId              : teams[transaction['items'][x]['fromTeamId']]
# player_name             : player_base[transaction['items'][x]['playerId']][0]
# player_position         : player_base[transaction['items'][x]['playerId']][1]
# toTeamId                : teams[transaction['items'][x]['toTeamId']]
# related_transaction_id  : transaction['relatedTransactionId']

data = {'transactionId': [],
        'year': [], 
        'scoringPeriod': [],
        'proposedDate': [],
        'processedDate': [],
        'executionType': [],
        'transactionType': [],
        'teamOwner': [],
        'status': [],
        'bidAmount': [],
        'itemType': [],
        'fromTeam': [],
        'playerName': [],
        'playerPosition': [],
        'toTeam': [],
        'relatedTransactionId': []}

class Transaction:
    def __init__(self, transactionId=None, year=None, scoringPeriod=None):
        self.transactionId = transactionId
        self.year = year
        self.scoringPeriod = scoringPeriod
        self.proposedDate = None
        self.processedDate = None
        self.executionType = None
        self.teamOwner = None
        self.status = None
        self.bidAmount = None
        self.itemType = None
        self.fromTeam = None
        self.playerName = None
        self.playerPosition = None
        self.toTeam = None
        self.relatedTransactionId = None

def processTransaction(trans):
    data['transactionId'].append(trans.transactionId)
    data['year'].append(trans.year)
    data['scoringPeriod'].append(trans.scoringPeriod)
    data['proposedDate'].append(trans.proposedDate)
    data['processedDate'].append(trans.processedDate)
    data['executionType'].append(trans.executionType)
    data['transactionType'].append(trans.transactionType)
    data['teamOwner'].append(trans.teamOwner)
    data['status'].append(trans.status)
    data['bidAmount'].append(trans.bidAmount)
    data['itemType'].append(trans.itemType)
    data['fromTeam'].append(trans.fromTeam)
    data['playerName'].append(trans.playerName)
    data['playerPosition'].append(trans.playerPosition)
    data['toTeam'].append(trans.toTeam)
    data['relatedTransactionId'].append(trans.relatedTransactionId)

In [8]:
years = list(range(2018, 2023))
# years = [2018]
scoring_periods = list(range(0, 19))

In [9]:
for year in years:
    player_base = get_player_base(year)
    
    url = f'https://fantasy.espn.com/apis/v3/games/ffl/seasons/{year}/segments/0/leagues/{league_id}'
    cookies = {"swid":      cookie_swid,
               "espn_s2":   cookie_espn_s2}

    for scoring_period in scoring_periods:
        params = {"scoringPeriodId": scoring_period, 
                  "view": ["mDraftDetail", "mStatus", "mSettings", "mTeam", "mTransactions2", "modular", "mNav"]}
        response = requests.get(url, params=params, cookies=cookies)
        if response.status_code != 200:
            print(response.json)
            continue

        members = {}
        for member in response.json()['members']:
            members[member['id']] = f"{member['firstName'].strip()} {member['lastName'].strip()}"

        teams = {}
        for team in response.json()['teams']:
            teams[team['id']] = teams[team['primaryOwner']] = {'league_id': team['id'], 
                                                               'primary_owner_id': team['primaryOwner'], 
                                                               'owner_name': members[team['primaryOwner']], 
                                                               'team_name': f"{team['location']} {team['nickname']}"}


        if 'transactions' not in response.json():
            new_transaction = Transaction(year, scoring_period)
            continue

        for trans in response.json()['transactions']:
            if 'items' in trans:
                for item in trans['items']:
                    new_transaction = Transaction(trans['id'], year, scoring_period)
                    new_transaction.executionType = trans['executionType']
                    new_transaction.transactionType = trans['type']
                    new_transaction.bidAmount = trans['bidAmount']
                    new_transaction.proposedDate = datetime.fromtimestamp(trans['proposedDate'] / 1000)

                    if 'processDate' in trans:
                        new_transaction.processedDate = datetime.fromtimestamp(trans['processDate'] / 1000)

                    if 'relatedTransactionId' in trans:
                        new_transaction.relatedTransactionId = trans['relatedTransactionId']

                    if 'status' in trans:
                        new_transaction.status = trans['status']

                    if 'teamId' in trans:
                        if trans['teamId'] in teams:
                            new_transaction.teamOwner = teams[trans['teamId']]['owner_name']
                        else:
                            new_transaction.teamOwner = 'Free Agency'

                    new_transaction.itemType = item['type']

                    if item['fromTeamId'] in teams:
                        new_transaction.fromTeam = teams[item['fromTeamId']]['owner_name']
                    else:
                        new_transaction.fromTeam = 'Free Agency'

                    if item['toTeamId'] in teams:
                        new_transaction.toTeam = teams[item['toTeamId']]['owner_name']
                    else:
                        new_transaction.toTeam = 'Free Agency'
                    
                    if new_transaction.teamOwner == 'Free Agency':
                        if new_transaction.fromTeam != 'Free Agency':
                            new_transaction.teamOwner = new_transaction.fromTeam
                        elif new_transaction.toTeam != 'Free Agency':
                            new_transaction.teamOwner = new_transaction.toTeam
                    
                    new_transaction.playerName, new_transaction.playerPosition = player_base[item['playerId']]
                    
                    processTransaction(new_transaction)
            else:
                new_transaction = Transaction(trans['id'], year, scoring_period)
                new_transaction.executionType = trans['executionType']
                new_transaction.transactionType = trans['type']
                new_transaction.bidAmount = trans['bidAmount']
                new_transaction.proposedDate = datetime.fromtimestamp(trans['proposedDate'] / 1000)

                if 'status' in trans:
                    new_transaction.status = trans['status']

                if 'teamId' in trans:
                    new_transaction.teamOwner = teams[trans['teamId']]['owner_name']

                if 'processedDate' in trans:
                    new_transaction.processedDate = datetime.fromtimestamp(trans['processedDate'] / 1000)

                if 'relatedTransactionId' in trans:
                    new_transaction.relatedTransactionId = trans['relatedTransactionId']
                
                processTransaction(new_transaction)

https://fantasy.espn.com/apis/v3/games/ffl/seasons/2018/players?view=players_wl
https://fantasy.espn.com/apis/v3/games/ffl/seasons/2019/players?view=players_wl
https://fantasy.espn.com/apis/v3/games/ffl/seasons/2020/players?view=players_wl
https://fantasy.espn.com/apis/v3/games/ffl/seasons/2021/players?view=players_wl
https://fantasy.espn.com/apis/v3/games/ffl/seasons/2022/players?view=players_wl


In [10]:
df = pd.DataFrame(data=data)

In [11]:
df.to_csv('ff-offers-mc.csv')
# df